In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import re
from artefacts import DATASETS_PATH, GLOBAL_CLIMATE_CHANGE_LITIGATION_PATH
import pandas as pd
import re
from tqdm import tqdm

from langchain.llms import HuggingFacePipeline
from langchain.chains import create_extraction_chain
from langchain.llms import LlamaCpp

from langchain.chat_models import ChatAnthropic, ChatOllama

# from huggingface_hub import login
# login()

# https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF


In [3]:
from ctransformers import AutoModelForCausalLM

MAX_CONTEXT_LENGTH = 4096

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
# llm = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7b-Chat-GGUF", model_file="llama-2-7b-chat.q4_K_M.gguf", model_type="llama", gpu_layers=30, context_length=MAX_CONTEXT_LENGTH)

llm = LlamaCpp(
    model_path="/home/clem/models/TheBloke/llama-2-7b-chat.q4_K_M.gguf",
    n_ctx=MAX_CONTEXT_LENGTH,
    n_gpu_layers=-1,
    n_batch=512, # Number of tokens to process in parallel Should be a number between 1 and n_ctx
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /home/clem/models/TheBloke/llama-2-7b-chat.q4_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4_K     [  

In [ ]:
llm("Question: In bash, how do I list all the text files in the current directory that have been modified in the last month? Answer:")

# [Llama-2-7b-Chat-GPTQ](https://huggingface.co/TheBloke/Llama-2-7b-Chat-GPTQ)

In [2]:
model_name = "TheBloke/Llama-2-7b-Chat-GPTQ"

# Use a [fast Rust-based tokenizer](https://huggingface.co/docs/tokenizers/index) if it is supported for
# a given model. If a fast tokenizer is not available for a given model, a normal Python-based tokenizer
# is returned instead.
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name, revision="main", trust_remote_code=False, device_map="auto")

pipe_llm = pipeline(
    "text-generation", # text2text-generation,  text-generation, summarization
    model=model,
    tokenizer=tokenizer
)

local_llm = HuggingFacePipeline(pipeline=pipe_llm)

# Ollama

In [5]:
ollama = ChatOllama(model="llama2:7b-chat")

# Information extraction

In [6]:
schema = {
    "properties": {
        "name": {"type": "string"},
        "height": {"type": "integer"},
        "hair_color": {"type": "string"},
    },
    "required": ["name", "height"],
}

# Input 
inp = """Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde."""

chain = create_extraction_chain(schema, ollama)

chain.run(inp)

 ########## 
<class 'langchain.schema.output.ChatGeneration'>


OutputParserException: Could not parse function call: 'function_call'